In [101]:
import serial

In [102]:
cmd_map = dict()
cmd_map["frequency"]="f"
cmd_map["wave"]="w"
cmd_map["duty_cycle"]="d"
cmd_map["offset"]="o"
cmd_map["phase"]="p"
cmd_map["atten"]="y"
cmd_map["amplitude"]="a"
cmd_map["on"]="b"

In [163]:
class Channel(object):
    def __init__(self, dds, num):
        self.dds = dds
        self.num = num
        
    @property
    def frequency(self):
        raw_value = self._frequency
        return float(raw_value)/100
        
    @frequency.setter
    def frequency(self, value):
        raw_value = int(value*100)
        self.dds._set(self, "frequency", raw_value)
        
    @property
    def wave(self):
        raw_value = self._wave
        return raw_value
        
    @property
    def duty_cycle(self):
        raw_value = self._duty_cycle
        return float(raw_value)/10
    
    @duty_cycle.setter
    def duty_cycle(self, value):
        raw_value = int(value*10)
        self.dds._set(self, "duty_cycle", raw_value)
        
    @property
    def offset(self):
        raw_value = self._offset
        return raw_value - 120
    
    @property
    def phase(self):
        raw_value = self._phase
        return self._phase
    
    @property
    def atten(self):
        raw_value = self._atten
        return raw_value   
    
    @property
    def on(self):
        raw_value = self._on
        return raw_value
    
    @property
    def amplitude(self):
        raw_value = self._amplitude
        return float(raw_value)/100
    
    @amplitude.setter
    def amplitude(self, value):
        raw_value = int(value*100)
        self.dds._set(self, "amplitude", raw_value)
        
    def __str__(self):
        return "{}".format(self.num)
    
    def __repr__(self):
        return "Channel<{}>".format(self.num)
        
def getter_gen(parameter):
    def getter_fcn(self):
        cmd = cmd_map[parameter]
        raw_value = self.dds._read(self, parameter)
        value = raw_value.split(cmd)[1]
        return int(value)
        
    return getter_fcn

def setter_gen(parameter):
    def setter_fcn(self, value):
        return self.dds._set(self, parameter, value)
        
    return setter_fcn

for attribute, _ in cmd_map.items():
    setattr(Channel, "_{}".format(attribute), property(
        getter_gen(attribute),
        setter_gen(attribute),))

In [164]:
class MHS5200A(object):
    def __init__(self, port="/dev/ttyUSB2"):
        cfg=dict()
        cfg["port"]=port
        cfg["baudrate"]=57600
        cfg["xonxoff"]=False        
        cfg["timeout"]=0.5
        cfg["rtscts"]=True
        cfg["dsrdtr"]=False
        self.cfg = cfg
        self.serial = serial.Serial(**cfg)
        self.channels = list()
        self.channels.append(Channel(self, 1))    
        self.channels.append(Channel(self, 2))        
        self.send("")
        
    def send(self, msg="", return_line = True):
        self.serial.flushInput()
        self.serial.flushOutput()
        cmd_str = ":{}\r\n".format(msg)
        self.serial.write(cmd_str.encode())
        if return_line:
            data = self.serial.readline()
            data_clean = data.decode().strip()
            return data_clean
        
    def _read(self, channel, prop):
        cmd_str = "r{}{}".format(channel, cmd_map[prop])
        return self.send(cmd_str, return_line=True)
    
    def _set(self, channel, prop, value):
        cmd_str = "s{}{}{}".format(channel, cmd_map[prop], value)
        response = self.send(cmd_str, return_line=True)
        assert(response=="ok")
        
    def save(self, memory_slot="0"):
        response = self.send("s{}u".format(memory_slot))
        assert(response=="ok")

    def load(self, memory_slot="0"):
        response = self.send("s{}v".format(memory_slot))
        assert(response=="ok")
        
    @property
    def model(self):
        return self.send("r0c")

sg = MHS5200A(port="/dev/ttyUSB5")
chan1 = sg.channels[0]
chan2 = sg.channels[1]

SerialException: [Errno 2] could not open port /dev/ttyUSB5: [Errno 2] No such file or directory: '/dev/ttyUSB5'

In [117]:
chan1.frequency

10.0

In [154]:
chan1.frequency = 10

In [157]:
sg.save(0)

In [158]:
chan1.frequency = 20
sg.save(1)

In [159]:
sg.load(0)

In [160]:
sg.send(":r6b")

':r6b0'

In [161]:
sg.send(":r1m")

':r0m'

In [162]:
sg.send(":s5m")

'ok'

In [119]:
chan1.frequency = 10

In [120]:
chan1.amplitude

10.0

In [121]:
chan1.amplitude=10

In [122]:
chan2.frequency=2

In [123]:
chan2.offset

0

In [124]:
chan2.on

1

In [125]:
sg.send("s1b1")

'ok'

In [126]:
sg.send("r1b")

':r1b1'

In [127]:
sg.send("r2b")

':r2b1'

In [128]:
sg.send("r3b")

':r3b0'

In [129]:
sg.send("r4b")

':r4b0'

In [130]:
chan1.wave

0

In [131]:
chan1._wave=1

In [132]:
chan1._duty_cycle

500

In [133]:
chan1._duty_cycle=100

In [134]:
chan1.duty_cycle=50

In [135]:
chan1.frequency=5

In [136]:
sg.send("s3f100")

'ok'

In [137]:
sg.send("s4f1000")

'ok'

In [138]:
sg.send("s5t10")

'ok'

In [139]:
sg.send("s8b0")

'ok'

In [140]:
sg.send(":s1b1")

'ok'

In [141]:
sg.send(":s3b1")

'ok'

In [142]:
sg.send(":r0e")

':r0e0000000000'

In [149]:
chan1.frequency=5

In [150]:
sg.send("s0u")

'ok'